# 🖊️ Handwriting Detection & OCR Pipeline - Implementation Plan

## RTX 3050 Optimized (8GB VRAM) | Local Execution Only

---

## 📋 MASTER IMPLEMENTATION PROMPT

This notebook contains the complete implementation plan and instructions for building a handwriting detection and OCR pipeline. When code generation is requested, follow these specifications exactly.

---

## 1. PROJECT OVERVIEW

**Goal:** Build a two-stage pipeline:
1. **Stage 1 - Detection:** YOLOv8 to detect handwritten text regions (words/lines)
2. **Stage 2 - OCR:** TrOCR (or fallback EasyOCR) to recognize text in detected regions

**Target Hardware:** NVIDIA RTX 3050 (8GB VRAM)
**Execution:** 100% local (no Colab, no cloud)

---

## 2. STRICT CONSTRAINTS CHECKLIST

| # | Constraint | Implementation |
|---|------------|----------------|
| 1 | **ONLY real handwritten data** | IAM, ICDAR, Bentham - NO synthetic fonts |
| 2 | **8GB VRAM limit** | imgsz=640, batch=2-8, AMP enabled |
| 3 | **Local execution only** | No Colab, no Drive, local paths only |
| 4 | **Deterministic conversion** | Scripted YOLO format conversion |
| 5 | **Detection + OCR pipeline** | YOLOv8 → TrOCR/EasyOCR |
| 6 | **Checkpoint & resume** | Clear save/load instructions |
| 7 | **Evaluation metrics** | mAP@0.5, mAP@0.5:0.95, CER, WER |
| 8 | **CLI runnable** | `python script.py --config ...` |
| 9 | **Quick-test mode** | 100 images, few epochs for validation |
| 10 | **Troubleshooting** | OOM, CUDA, driver guidance |

## 3. RTX 3050 HYPERPARAMETER DEFAULTS

### Critical GPU-Memory Settings

```yaml
# YOLO Detection Training Defaults
imgsz: 640                    # Image size (reduce to 512 if OOM)
batch: 4                      # Start with 4, reduce to 2 if OOM
device: 0                     # GPU device ID
amp: true                     # Mixed precision - CRITICAL for 3050
workers: 4                    # DataLoader workers (reduce if RAM issues)
cache: false                  # Don't cache images in RAM
epochs: 50                    # Adjust based on dataset size
patience: 10                  # Early stopping patience

# Gradient Accumulation (simulate larger batch)
accumulate: 4                 # Effective batch = batch * accumulate = 16

# Memory-Critical Fallbacks
# If OOM persists:
#   1. batch: 2
#   2. imgsz: 512
#   3. workers: 2
#   4. Close other GPU applications

# OCR (TrOCR) Settings
ocr_batch_size: 8             # Process crops in batches
ocr_max_length: 128           # Max sequence length
use_fp16: true                # Half precision inference
```

### Memory Estimation for RTX 3050 (8GB)
| Config | Estimated VRAM Usage |
|--------|---------------------|
| YOLOv8n, batch=4, imgsz=640 | ~3-4 GB |
| YOLOv8s, batch=2, imgsz=640 | ~4-5 GB |
| TrOCR inference, batch=8 | ~2-3 GB |
| Combined pipeline inference | ~4-5 GB |

## 4. DATASET REQUIREMENTS & ACCESS

### ⚠️ CRITICAL: Real Handwritten Data ONLY

**Allowed Datasets (in order of recommendation):**

### A. IAM Handwriting Database (PRIMARY RECOMMENDATION)
- **URL:** https://fki.tic.heia-fr.ch/databases/iam-handwriting-database
- **Access:** FREE registration required
- **Sign-up Steps:**
  1. Go to https://fki.tic.heia-fr.ch/databases/iam-handwriting-database
  2. Click "Register" and create an account
  3. After email verification, request access to IAM dataset
  4. Download: `words.tgz`, `lines.tgz`, `ascii.tgz` (annotations)
- **Content:** ~115,000 word images from 657 writers
- **Format:** Word/line-level segmented images with ground truth text
- **Best for:** Both detection training and OCR

### B. ICDAR Handwriting Recognition Datasets
- **URL:** https://rrc.cvc.uab.es/
- **Access:** Registration required
- **Datasets:**
  - ICDAR 2013 Handwriting Segmentation
  - ICDAR 2017 Handwritten Document Recognition
- **Content:** Document images with word/line bounding boxes

### C. CVL Database
- **URL:** https://cvl.tuwien.ac.at/research/cvl-databases/an-off-line-database-for-writer-retrieval-writer-identification-and-word-spotting/
- **Access:** FREE, direct download
- **Content:** 310 documents from 27 writers

### D. Bentham Papers (HTR Dataset)
- **URL:** https://transcriptorium.eu/datasets/bentham-collection/
- **Access:** FREE
- **Content:** Historical handwritten documents

### Dataset Directory Structure (Expected)
```
./data/
├── iam/
│   ├── words/           # Word images
│   ├── lines/           # Line images  
│   ├── ascii/           # Ground truth annotations
│   └── splits/          # train/val/test splits
├── yolo_format/
│   ├── images/
│   │   ├── train/
│   │   └── val/
│   ├── labels/
│   │   ├── train/
│   │   └── val/
│   └── data.yaml
└── ocr_crops/           # Cropped regions for OCR training
```

### 🚫 FORBIDDEN Data Sources
- Font-rendered "handwriting" fonts
- Synthetic text generators (SynthText, etc.)
- Printed text datasets (unless for negative samples only)
- AI-generated handwriting images

## 5. ENVIRONMENT SETUP SPECIFICATIONS

### Required Software Versions (RTX 3050 Compatible)

```bash
# CUDA Compatibility for RTX 3050 (Ampere architecture)
CUDA Toolkit: 11.8 or 12.1 (recommended)
cuDNN: 8.6+ (compatible with CUDA version)
NVIDIA Driver: 520+ (for CUDA 11.8) or 530+ (for CUDA 12.1)

# Python Environment
Python: 3.10.x (recommended) or 3.11.x
```

### Exact Package Versions (Tested)

```bash
# Create conda environment
conda create -n handwriting python=3.10 -y
conda activate handwriting

# PyTorch with CUDA 11.8 (RTX 3050 compatible)
pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

# Alternative: CUDA 12.1
# pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121

# Core packages
pip install ultralytics==8.0.200          # YOLOv8
pip install transformers==4.35.0          # TrOCR
pip install easyocr==1.7.1                # Backup OCR
pip install paddleocr==2.7.0.3            # Alternative OCR (optional)

# Utilities
pip install opencv-python==4.8.1.78
pip install Pillow==10.1.0
pip install matplotlib==3.8.0
pip install pandas==2.1.1
pip install numpy==1.24.3
pip install tqdm==4.66.1
pip install editdistance==0.6.2           # For CER/WER calculation
pip install jiwer==3.0.3                  # WER calculation
pip install pycocotools==2.0.7            # mAP calculation
pip install pyyaml==6.0.1
pip install albumentations==1.3.1         # Data augmentation
```

### Verify Installation Script
```python
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Version: {torch.version.cuda}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
```

## 6. ARTIFACTS TO BE PRODUCED

### Checklist of Deliverables

| # | Artifact | File | Description |
|---|----------|------|-------------|
| **A** | README | `README.md` | Complete setup guide, commands, troubleshooting |
| **B1** | Environment Check | `scripts/check_environment.py` | Verify CUDA, GPU, packages |
| **B2** | Dataset Downloader | `scripts/download_dataset.py` | Instructions + manifest checker |
| **B3** | YOLO Converter | `scripts/convert_to_yolo.py` | IAM → YOLO format conversion |
| **B4** | Data Validator | `scripts/validate_dataset.py` | Check labels, visualize samples |
| **B5** | Detection Trainer | `scripts/train_detector.py` | YOLOv8 training with RTX 3050 config |
| **B6** | Inference Script | `scripts/run_inference.py` | Detect regions, save crops + JSON |
| **B7** | OCR Runner | `scripts/run_ocr.py` | TrOCR/EasyOCR on crops |
| **B8** | Post-processor | `scripts/postprocess.py` | Box ordering, line merging |
| **B9** | Evaluator | `scripts/evaluate.py` | mAP, CER, WER computation |
| **B10** | Full Pipeline | `scripts/predict.py` | End-to-end `predict(image_path)` → JSON |
| **B11** | Visualizer | `scripts/visualize.py` | Draw boxes + OCR overlay |
| **C** | Quick Test | `scripts/quick_test.py` | 100 images, smoke test mode |
| **D** | Config | `configs/rtx3050.yaml` | Optimized hyperparameters |
| **E** | Notebook | `main.ipynb` | Interactive version with all cells |

### Directory Structure (Final)
```
handwriting/
├── README.md
├── requirements.txt
├── configs/
│   ├── rtx3050.yaml          # RTX 3050 optimized config
│   ├── quick_test.yaml       # Minimal config for testing
│   └── data.yaml             # YOLO dataset config
├── scripts/
│   ├── check_environment.py
│   ├── download_dataset.py
│   ├── convert_to_yolo.py
│   ├── validate_dataset.py
│   ├── train_detector.py
│   ├── run_inference.py
│   ├── run_ocr.py
│   ├── postprocess.py
│   ├── evaluate.py
│   ├── predict.py
│   ├── visualize.py
│   └── quick_test.py
├── data/
│   ├── iam/                  # Raw IAM dataset (user downloads)
│   └── yolo_format/          # Converted YOLO format
├── models/
│   ├── detector/             # YOLO checkpoints
│   └── ocr/                  # TrOCR cache
├── outputs/
│   ├── detections/           # Detection results
│   ├── crops/                # Cropped regions
│   ├── visualizations/       # Annotated images
│   └── results/              # Evaluation metrics
└── main.ipynb                # Interactive notebook
```

## 7. DETAILED IMPLEMENTATION SPECIFICATIONS

### 7.1 Dataset Conversion (IAM → YOLO)

**Input:** IAM dataset structure with XML annotations
**Output:** YOLO format with normalized coordinates

```
YOLO Label Format (per line):
<class_id> <x_center> <y_center> <width> <height>

Where:
- class_id: 0 (single class: "handwriting")
- All coordinates normalized to [0, 1] relative to image size
```

**Conversion Logic:**
1. Parse IAM XML annotations for word/line bounding boxes
2. Convert absolute pixel coords → normalized coords
3. Handle edge cases: boxes outside image bounds, zero-size boxes
4. Split into train/val/test following IAM official splits
5. Generate `data.yaml` for Ultralytics

**Validation Checks:**
- All labels have 5 values per line
- All values in [0, 1] range
- No empty label files
- Image-label filename correspondence

---

### 7.2 YOLO Detection Training

**Model:** YOLOv8n or YOLOv8s (nano/small for 3050)

**Training Command Template:**
```bash
python train_detector.py \
    --data configs/data.yaml \
    --model yolov8n.pt \
    --imgsz 640 \
    --batch 4 \
    --epochs 50 \
    --device 0 \
    --amp \
    --project models/detector \
    --name handwriting_v1
```

**Resume Training:**
```bash
python train_detector.py --resume models/detector/handwriting_v1/weights/last.pt
```

**Key Ultralytics Parameters:**
```python
model.train(
    data='configs/data.yaml',
    epochs=50,
    imgsz=640,
    batch=4,
    device=0,
    amp=True,                    # Mixed precision
    workers=4,
    patience=10,                 # Early stopping
    save=True,
    save_period=5,               # Checkpoint every 5 epochs
    cache=False,                 # Don't cache (save RAM)
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    box=7.5,                     # Box loss gain
    cls=0.5,                     # Class loss gain
    dfl=1.5,                     # Distribution focal loss
)
```

---

### 7.3 OCR Integration

**Primary: TrOCR (Microsoft)**
```python
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

# Handwriting-specific model
model_name = "microsoft/trocr-base-handwritten"
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

# Move to GPU with half precision
model = model.half().cuda()
```

**Fallback: EasyOCR**
```python
import easyocr
reader = easyocr.Reader(['en'], gpu=True)
```

**OCR Decision Tree:**
1. Try TrOCR first (better handwriting accuracy)
2. If OOM: reduce batch size or use EasyOCR
3. If TrOCR too slow: use EasyOCR for quick tests

**TrOCR Fine-tuning Note (3050 Limitation):**
- Full fine-tuning NOT recommended on 3050 (memory intensive)
- Alternative: Use pretrained model OR fine-tune with:
  - LoRA adapters (reduced memory)
  - Frozen encoder, train decoder only
  - Very small batch (1-2) with gradient accumulation

### 7.4 Post-Processing & Box Ordering

**Box Ordering Algorithm:**
```
1. Sort boxes by y-coordinate (top to bottom)
2. Group boxes into lines (y-overlap threshold)
3. Within each line, sort by x-coordinate (left to right)
4. Merge adjacent boxes if spacing < threshold
5. Concatenate OCR results following order
```

**Line Detection Heuristic:**
```python
def group_into_lines(boxes, y_threshold=20):
    """Group boxes into text lines based on vertical overlap."""
    sorted_boxes = sorted(boxes, key=lambda b: b['y1'])
    lines = []
    current_line = [sorted_boxes[0]]
    
    for box in sorted_boxes[1:]:
        if abs(box['y1'] - current_line[-1]['y1']) < y_threshold:
            current_line.append(box)
        else:
            lines.append(sorted(current_line, key=lambda b: b['x1']))
            current_line = [box]
    lines.append(sorted(current_line, key=lambda b: b['x1']))
    return lines
```

---

### 7.5 Evaluation Metrics

**Detection Metrics:**
```python
# Using Ultralytics built-in validation
from ultralytics import YOLO
model = YOLO('path/to/best.pt')
metrics = model.val(data='configs/data.yaml')

# Key metrics:
# - mAP@0.5: Mean Average Precision at IoU=0.5
# - mAP@0.5:0.95: Mean AP averaged over IoU 0.5 to 0.95
# - Precision, Recall, F1
```

**OCR Metrics:**
```python
from jiwer import wer, cer

def calculate_metrics(predictions, ground_truths):
    """Calculate CER and WER for OCR results."""
    total_cer = cer(ground_truths, predictions)
    total_wer = wer(ground_truths, predictions)
    return {'CER': total_cer, 'WER': total_wer}

# Expected ranges (good model):
# CER: 5-15% (character error rate)
# WER: 10-30% (word error rate)
# Note: Handwriting is harder than printed text
```

**Expected Metrics on IAM (reference):**
| Model | mAP@0.5 | CER | WER |
|-------|---------|-----|-----|
| YOLOv8n (detection) | 85-92% | - | - |
| TrOCR (pretrained) | - | 8-15% | 20-35% |
| TrOCR (fine-tuned) | - | 4-8% | 12-20% |

---

### 7.6 Output JSON Format

```json
{
  "image_path": "/path/to/image.jpg",
  "image_size": {"width": 1200, "height": 800},
  "detections": [
    {
      "id": 0,
      "box": [x1, y1, x2, y2],
      "confidence": 0.92,
      "ocr_text": "Hello",
      "ocr_confidence": 0.87,
      "line_id": 0
    },
    {
      "id": 1,
      "box": [x1, y1, x2, y2],
      "confidence": 0.89,
      "ocr_text": "World",
      "ocr_confidence": 0.91,
      "line_id": 0
    }
  ],
  "lines": [
    {"line_id": 0, "text": "Hello World", "boxes": [0, 1]}
  ],
  "aggregated_text": "Hello World\nNext line here...",
  "processing_time_ms": 245
}
```

## 8. TROUBLESHOOTING FAQ

### 🔴 Out of Memory (OOM) Errors

**Symptoms:**
```
RuntimeError: CUDA out of memory. Tried to allocate X MiB
torch.cuda.OutOfMemoryError
```

**Solutions (in order):**
1. **Reduce batch size:** `--batch 2` or even `--batch 1`
2. **Reduce image size:** `--imgsz 512` or `--imgsz 480`
3. **Enable AMP:** Ensure `--amp` flag is set (uses FP16)
4. **Gradient accumulation:** Set `accumulate=4` to simulate larger batch
5. **Clear cache:** Add `torch.cuda.empty_cache()` between operations
6. **Close other apps:** Check `nvidia-smi` for other GPU processes
7. **Reduce workers:** `--workers 2` or `--workers 0`

**For TrOCR specifically:**
- Use `model.half()` for FP16 inference
- Process crops in small batches (4-8)
- Use EasyOCR as fallback (lighter)

---

### 🔴 CUDA/Driver Issues

**Check CUDA Status:**
```bash
# Check NVIDIA driver
nvidia-smi

# Check CUDA toolkit
nvcc --version

# Check PyTorch CUDA
python -c "import torch; print(torch.cuda.is_available())"
python -c "import torch; print(torch.version.cuda)"
```

**Common Issues:**

1. **`torch.cuda.is_available()` returns False:**
   - Install CUDA-enabled PyTorch: 
     ```bash
     pip uninstall torch torchvision torchaudio
     pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
     ```
   - Verify driver supports CUDA version

2. **CUDA version mismatch:**
   - PyTorch CUDA version must be ≤ system CUDA version
   - RTX 3050 supports CUDA 11.x and 12.x
   - Recommended: CUDA 11.8 with PyTorch 2.1.0

3. **Driver too old:**
   - RTX 3050 needs driver ≥ 470 (CUDA 11.4+)
   - Recommended: driver 520+ for CUDA 11.8
   - Update from: https://www.nvidia.com/drivers

---

### 🔴 Dataset Access Problems

**IAM Database:**
- Registration required (free)
- Account approval may take 24-48 hours
- If denied: email administrators with research purpose

**ICDAR datasets:**
- Some competitions have closed access
- Check https://rrc.cvc.uab.es/ for open datasets

**Workaround:**
- Use CVL database (direct download, no registration)
- Use a small local dataset for pipeline validation

---

### 🔴 Slow Training

**Expected times on RTX 3050:**
| Dataset Size | Epochs | Estimated Time |
|--------------|--------|----------------|
| 1,000 images | 50 | ~30-45 min |
| 10,000 images | 50 | ~4-6 hours |
| 50,000 images | 50 | ~20-30 hours |

**Speed optimizations:**
- Use `--cache ram` if you have 32GB+ RAM
- Use `--workers 4` (not more, diminishing returns)
- Use SSD for dataset storage
- Disable validation during training: `--val false` (not recommended)

---

### 🔴 Poor Detection/OCR Results

**Detection issues:**
- Increase training epochs
- Try YOLOv8s instead of YOLOv8n
- Add data augmentation (already default in Ultralytics)
- Check label quality: run `validate_dataset.py`

**OCR issues:**
- Ensure crops are not too small (min 32px height)
- Add padding around crops (10-20%)
- Try different TrOCR variants:
  - `microsoft/trocr-base-handwritten` (general)
  - `microsoft/trocr-large-handwritten` (better but heavier)
- Preprocess crops: grayscale, contrast enhancement

## 9. QUICK-TEST MODE SPECIFICATION

### Purpose
Validate the entire pipeline works on RTX 3050 before full training.

### Quick-Test Parameters
```yaml
# Quick test config
mode: quick_test
num_samples: 100          # Only 100 images
train_samples: 80
val_samples: 20
epochs: 5                 # Minimal epochs
batch_size: 2             # Conservative
imgsz: 640
patience: 3               # Early stopping
```

### Expected Quick-Test Results
| Metric | Expected Range | Notes |
|--------|---------------|-------|
| Training time | 5-10 min | YOLOv8n, 5 epochs |
| mAP@0.5 | 30-60% | Low due to few samples |
| Inference time | 50-100ms/image | Detection only |
| OCR time | 100-200ms/crop | TrOCR batch=8 |
| Total pipeline | 500-1000ms/image | Full page, ~10 words |

### Quick-Test Command
```bash
python scripts/quick_test.py --samples 100 --epochs 5 --batch 2
```

### Success Criteria
✅ No OOM errors
✅ Training completes all epochs  
✅ Detection produces valid boxes
✅ OCR returns text strings
✅ JSON output is well-formed
✅ Visualization images are created

## 10. FUTURE IMPROVEMENTS & NEXT STEPS

### 10.1 Data Augmentation Strategies
```python
# Recommended augmentations for handwriting
augmentations = {
    'geometric': [
        'RandomRotate(limit=5)',      # Slight rotation
        'ShiftScaleRotate',           # Perspective changes
        'ElasticTransform(alpha=50)', # Mimic writing variations
    ],
    'photometric': [
        'RandomBrightnessContrast',
        'GaussNoise',
        'RandomGamma',
        'CLAHE',                      # Contrast enhancement
    ],
    'handwriting_specific': [
        'RandomErase',                # Simulate ink gaps
        'Blur',                       # Simulate scan quality
        'JpegCompression',            # Realistic artifacts
    ]
}
```

### 10.2 TrOCR Fine-tuning (When Resources Allow)

**Option A: LoRA Fine-tuning (3050 Compatible)**
```python
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,                    # Rank
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
)
model = get_peft_model(model, lora_config)
# Reduces trainable params by ~90%
```

**Option B: Decoder-only Fine-tuning**
```python
# Freeze encoder
for param in model.encoder.parameters():
    param.requires_grad = False
# Only train decoder (smaller memory footprint)
```

### 10.3 End-to-End Alternatives (Research Directions)

**A. TrOCR Direct (No Detection)**
- For single-line images
- Feed entire line to TrOCR
- Simpler but requires pre-segmented data

**B. DTROCR / Donut**
- Document understanding transformers
- No explicit detection step
- Heavier, may not fit on 3050

**C. PaddleOCR PP-OCRv4**
- Integrated detection + recognition
- Optimized for edge deployment
- Good balance of speed/accuracy

**D. Segment-then-Recognize (SAM + OCR)**
- Use SAM for text region segmentation
- Apply OCR to segments
- Flexible but complex

### 10.4 Resource Pointers

| Topic | Resource |
|-------|----------|
| TrOCR Paper | https://arxiv.org/abs/2109.10282 |
| YOLOv8 Docs | https://docs.ultralytics.com |
| IAM Dataset | https://fki.tic.heia-fr.ch/databases/iam-handwriting-database |
| HuggingFace TrOCR | https://huggingface.co/microsoft/trocr-base-handwritten |
| PaddleOCR | https://github.com/PaddlePaddle/PaddleOCR |
| Handwriting Recognition Survey | https://arxiv.org/abs/2112.07917 |

## 11. EXECUTION WORKFLOW

### Step-by-Step Local Execution Guide

```
┌─────────────────────────────────────────────────────────────────┐
│                    EXECUTION FLOWCHART                          │
└─────────────────────────────────────────────────────────────────┘

Step 1: Environment Setup
    ├── Create conda environment
    ├── Install PyTorch + CUDA
    ├── Install dependencies
    └── Run: python scripts/check_environment.py
         ↓
Step 2: Dataset Preparation  
    ├── Download IAM dataset (manual, requires registration)
    ├── Place in ./data/iam/
    ├── Run: python scripts/convert_to_yolo.py
    └── Run: python scripts/validate_dataset.py
         ↓
Step 3: Quick Test (Recommended First)
    └── Run: python scripts/quick_test.py --samples 100 --epochs 5
         ↓
Step 4: Full Training
    ├── Run: python scripts/train_detector.py --config configs/rtx3050.yaml
    └── Monitor: tensorboard --logdir models/detector/
         ↓
Step 5: Inference & OCR
    ├── Run: python scripts/run_inference.py --source ./test_images/
    └── Run: python scripts/run_ocr.py --crops ./outputs/crops/
         ↓
Step 6: Evaluation
    ├── Run: python scripts/evaluate.py --predictions ./outputs/
    └── Review: ./outputs/results/metrics.json
         ↓
Step 7: Full Pipeline Usage
    └── python scripts/predict.py --image ./my_handwritten_doc.jpg
```

### CLI Commands Summary

```bash
# 1. Environment check
python scripts/check_environment.py

# 2. Convert dataset
python scripts/convert_to_yolo.py --input ./data/iam --output ./data/yolo_format

# 3. Validate dataset  
python scripts/validate_dataset.py --data ./data/yolo_format --visualize

# 4. Quick test
python scripts/quick_test.py --samples 100 --epochs 5

# 5. Train detector
python scripts/train_detector.py --config configs/rtx3050.yaml --epochs 50

# 6. Resume training
python scripts/train_detector.py --resume models/detector/handwriting_v1/weights/last.pt

# 7. Run inference
python scripts/run_inference.py --model models/detector/best.pt --source ./test_images/ --save-crops

# 8. Run OCR
python scripts/run_ocr.py --crops ./outputs/crops/ --model trocr --batch 8

# 9. Evaluate
python scripts/evaluate.py --det-results ./outputs/detections/ --ocr-results ./outputs/ocr/ --gt ./data/yolo_format/labels/val/

# 10. Full pipeline
python scripts/predict.py --image ./document.jpg --output ./result.json --visualize
```

## 12. IMPLEMENTATION SUMMARY & REMINDERS

---

### ⚠️ CRITICAL REMINDERS FOR CODE GENERATION

```
╔══════════════════════════════════════════════════════════════════╗
║  🚨 MANDATORY: REAL HANDWRITTEN DATA ONLY                        ║
║                                                                  ║
║  • IAM, ICDAR, CVL, Bentham = ✅ ALLOWED                         ║
║  • Synthetic fonts, printed text = ❌ FORBIDDEN                  ║
║  • AI-generated handwriting = ❌ FORBIDDEN                       ║
║                                                                  ║
║  If dataset unavailable: provide download instructions,          ║
║  NOT synthetic alternatives                                      ║
╚══════════════════════════════════════════════════════════════════╝
```

### GPU Memory Defaults (RTX 3050 - 8GB)

| Parameter | Default | Fallback (OOM) | Aggressive |
|-----------|---------|----------------|------------|
| `imgsz` | 640 | 512 | 480 |
| `batch` | 4 | 2 | 1 |
| `workers` | 4 | 2 | 0 |
| `amp` | True | True | True |
| `accumulate` | 4 | 8 | 16 |
| `cache` | False | False | False |

### Key File Paths (Configurable)

```python
# All paths should be configurable via CLI args or env vars
DATA_ROOT = os.getenv('HW_DATA_ROOT', './data')
MODEL_ROOT = os.getenv('HW_MODEL_ROOT', './models')
OUTPUT_ROOT = os.getenv('HW_OUTPUT_ROOT', './outputs')
```

---

## ✅ READY FOR CODE GENERATION

This implementation plan is complete. When you request the actual code, I will generate:

1. **All Python scripts** in `./scripts/` directory
2. **Configuration files** in `./configs/` directory  
3. **README.md** with full documentation
4. **requirements.txt** with exact versions
5. **Interactive notebook cells** for step-by-step execution

### To generate the code, say:
> "Generate the complete implementation code following this plan"

or for specific parts:
> "Generate only the dataset conversion script"
> "Generate only the training script"
> "Generate the full pipeline predict function"

---

*Plan created: December 2024*
*Target: NVIDIA RTX 3050 (8GB VRAM)*
*Primary dataset: IAM Handwriting Database*
*Architecture: YOLOv8 Detection → TrOCR Recognition*